In [1]:
from pathlib import Path
import pickle
from joblib import Parallel, delayed
from itertools import product
from smount_predictors.src.SeamountHelp import PipelinePredictor
from smount_predictors import SeamountHelp
import numpy as np
import pandas as pd
import simplekml
import xarray as xr


In [2]:
longitude_pairs = []
for lon in range(-180, 160, 20):
    longitude_pairs.append((lon, lon + 20))

latitude_pairs = []
for lat in range(-60, 70, 20):
    latitude_pairs.append((lat, lat + 20))


latlons = list(product(longitude_pairs, latitude_pairs))
latlons = latlons[:4]

In [5]:
model = pickle.load(open('out/cluster_tuned_model.pkl', 'rb'))
data_p = Path('data/vgg_swot.grd')

In [ ]:
redictions = pd.DataFrame(columns=['lon', 'lat', 'z', 'class', 'cluster'])

def predict_zone(zone):
    lon = zone[0]
    lat = zone[1]
    # print(f'Predicting zone {lon}-{lat}')
    data = SeamountHelp.readAndFilterGRD(data_p, lat, lon).to_dataframe().reset_index()
    zone_pred = model.predict(data[['lon', 'lat', 'z']])
    return zone_pred

predictions = Parallel(n_jobs=-1)(delayed(predict_zone)(zone) for zone in latlons)
predictions = pd.concat(predictions)
global_predictions = xr.Dataset.from_dataframe(predictions.set_index(['lon', 'lat']))
global_predictions.to_netcdf('out/global_predictions.nc')

In [ ]:
mounts = predictions.groupby('cluster').mean().reset_index()
kml = simplekml.Kml()
for i, row in mounts.iterrows():
    kml.newpoint(name=f'{row.cluster}', coords=[(row.lon, row.lat, row.z)])
kml.save('out/global_mounts.kml')

In [ ]:
! open out/global_mounts.kml